# Supervised Learning - Regression

### Lab 01

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from numpy.linalg import inv
from time import time
import random

In [3]:
def add_intercept_column(X_train, X_test):
    #Concatenating the intercept to training set as variable b of shape = (no. of samples used for training, 1)
    b=np.ones((X_train.shape[0],1))
    X_train = np.concatenate((b, X_train), axis=1)

    #doing the same for the testing set
    b=np.ones((X_test.shape[0],1))
    X_test=np.concatenate((b,X_test),axis=1)
    
    return X_train, X_test

### Linear Regression Class Definition

In [9]:
#method argument
#0 - default value - Batch Gradient Descent
#1 - Mini-Batch Gradient Descent
#2 - Stochastic Gradient Descent

#early stopping argument
#False - off
#True - on (Default)

class linearRegression:
    def __init__(self):
        self.X_test=[]
        self.X_train=[]
        self.y_test=[]
        self.y_train=[]
        self.theta=[]
        
    def fit_data(self, xtest, xtrain, ytest, ytrain):
        self.X_test=xtest
        self.X_train=xtrain
        self.y_test=ytest
        self.y_train=ytrain
        assert X_train.shape[0] == y_train.shape[0]
        self.theta=np.zeros(xtrain.shape[1])
        
    def grad_descent(self, method=0, early_stopping=True, tol=0.0001, alpha=0.0001):
        
        start = time()
        loss_old=10000
        iter_stop=0
        n_epochs=10000
        batch_size=10
        
        self.theta=np.zeros(self.X_train.shape[1])
        if (method == 0):
            #Batch Gradient Descent
            xtrain=self.X_train
            ytrain=self.y_train
        
        j=0
        used_indexes=[]
        
        for i in range(n_epochs):
            
            if (method == 1):
                #Mini-Batch Gradient Descent
                if (j>self.X_train.shape[0]-batch_size-1):
                    j=0
                xtrain=self.X_train[j:j+batch_size,:]
                ytrain=self.y_train[j:j+batch_size,]
                j+=batch_size
            elif (method == 2):
                #Stochastic Gradient Descent
                n=random.randint(0,self.X_train.shape[0]-1)
                while n in used_indexes:
                    n=random.randint(0,self.X_train.shape[0]-1)
                used_indexes.append(n)
                if (len(used_indexes)==self.X_train.shape[0]):
                    used_indexes=[]
                xtrain=self.X_train[n,:].reshape(1,-1)
                ytrain=np.array([self.y_train[n]])
                
            #print(ytrain)
            #h(theta) = y_hat = X . theta
            y_hat = self.h_theta(xtrain, self.theta)
            #print(y_hat)
            assert y_hat.shape == ytrain.shape
            
            iter_stop+=1
            if (early_stopping):
                #compute MSE (loss) and compare it to the old loss
                #if difference is below the tolerance value, then break
                loss_new = self.mse(y_hat, ytrain)
                loss_diff = abs(loss_new - loss_old)
                if (loss_diff <= tol):
                    break
                loss_old = loss_new

            #error = y_hat - y
            error = y_hat - ytrain

            #gradient = (h_theta - y) * X [Matrix Multiplication]
            grad = self.compute_gradient(xtrain, error)

            #theta = theta - alpha * gradient
            self.theta = self.theta - alpha * grad

        time_taken = time() - start
        print("Time taken to run the algorithm: {}".format( time_taken ) )
        print("Number of iterations run by the algorithm: {}".format( iter_stop ) )

    def predict(self):
        y_hat = self.h_theta(self.X_test, self.theta)
        #print(theta.shape[1])
        #y_hat.shape
        #theta.shape
        assert y_hat.shape == self.y_test.shape
        mean_sq_error = self.mse(y_hat, self.y_test)
        print("Calculated Mean Square Error is: ", mean_sq_error)
        return y_hat

    def h_theta(self, X,theta):
        return X @ theta

    def mse(self, y_hat, y):
        return ((y_hat - y)**2).sum() / y_hat.shape[0]

    def compute_gradient(self, X, error):
        return X.T @ error

In [5]:
#Loading Boston House Prices Data
boston = load_boston()
boston.feature_names
X=boston.data

#where m = number of samples (rows), n = number of features (columns)
m=X.shape[0]
n=X.shape[1]

#Loading target variable
y=boston.target
assert m==y.shape[0]

In [6]:
#Standardizing data, assuming Gaussion(normal) distribution of current data
scaler=StandardScaler()
X=scaler.fit_transform(X)

#Splitting data into test and train batches
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

#Calling add_intercept function to add the intercept column to test & train data
X_train, X_test = add_intercept_column(X_train, X_test)

In [10]:
#Instantiating the class linearRegression and calling relevant methods
model = linearRegression()
model.fit_data(X_test, X_train, y_test, y_train)
model.grad_descent(0,False)


Time taken to run the algorithm: 0.15270757675170898
Number of iterations run by the algorithm: 10000


In [11]:
model.predict()

Calculated Mean Square Error is:  26.161805928538673


array([18.76596093, 10.89430074, 21.91445566, 21.98496912, 19.19231075,
       19.83480258, 31.26251366, 24.89666478, 29.67174402, 25.96960685,
       28.72431433, 33.508277  , 20.83101727, 23.56408451, 16.14627059,
       30.60206329, 23.9112637 , 30.36686479, 21.03663104, 24.45191047,
       12.93330235, 25.41873334, 18.87812656, 19.97046498,  5.54161567,
       27.8352254 , 40.52576349, 32.32437992, 12.61835213, 27.30256405,
       16.64041394, 26.64382691, 21.95843415, 11.46627871, 32.7434173 ,
        7.62347408, 20.85623495, 38.20257033, 28.05013377, 22.58601329,
       20.97475336, 15.48519505, 27.56677181, 16.26459608, 31.75991491,
       22.49648993, 11.59331993, 42.54885842, 14.26107836, 16.9905738 ,
       23.84108129, 25.54540279, 34.88927516, 17.92537523, 12.76351629,
       16.92208089,  8.86251779, 22.09676996, 21.86449868,  3.67616066,
       23.23760784, 30.57758186, 33.66326245, 24.89771382, 22.00999166,
       24.1334458 , 25.86535485, 24.01323727, 13.24468958, 40.88

NameError: name 'mse' is not defined